In [15]:
using FastGaussQuadrature
using LinearAlgebra

N = 100 # 10点ガウス求積を使用(=> 零点が10個！)
a = 0.0 # 積分の下限
b = 1.0 # 積分の上限

# 積分点と重みを計算
x, w = gausslegendre(N)

# 積分区間の調整
function adjust_xw(x, w, a, b)
    @assert b > a
    x_adjusted = (b - a) * 0.5 * (x .+ 1) .+ a
    w_adjusted = (0.5 * (b - a)) .* w
    return x_adjusted, w_adjusted
end

adjust_xw (generic function with 1 method)

変更した積分区間などの確認:
f = 1
を積分すると、積分値が積分区間の幅になる

In [16]:
# 全てのデータが引数で渡されているか、チェックしておく
@code_warntype adjust_xw(x, w, a, b)

MethodInstance for adjust_xw(::Vector{Float64}, ::Vector{Float64}, ::Float64, ::Float64)
  from adjust_xw(x, w, a, b) in Main at In[15]:12
Arguments
  #self#::Core.Const(adjust_xw)
  x::Vector{Float64}
  w::Vector{Float64}
  a::Float64
  b::Float64
Locals
  w_adjusted::Vector{Float64}
  x_adjusted::Vector{Float64}
Body::Tuple{Vector{Float64}, Vector{Float64}}
1 ─       Core.NewvarNode(:(w_adjusted))
│         Core.NewvarNode(:(x_adjusted))
│   %3  = (b > a)::Bool
└──       goto #3 if not %3
2 ─       goto #4
3 ─ %6  = Base.AssertionError("b > a")::Core.PartialStruct(AssertionError, Any[String])
└──       Base.throw(%6)
4 ┄ %8  = (b - a)::Float64
│   %9  = Base.broadcasted(Main.:+, x, 1)::Core.PartialStruct(Base.Broadcast.Broadcasted{Base.Broadcast.DefaultArrayStyle{1}, Nothing, typeof(+), Tuple{Vector{Float64}, Int64}}, Any[Core.Const(+), Core.PartialStruct(Tuple{Vector{Float64}, Int64}, Any[Vector{Float64}, Core.Const(1)]), Core.Const(nothing)])
│   %10 = Base.materialize(%9)::Vector{

In [17]:
# 簡単なテストを実行

let # x_, w_が名前空間を汚さないようにスコープを作る
    # Test case: [-1, 1]
    x_, w_ = adjust_xw(x, w, -1.0, 1.0) 
    @assert x_ ≈ x
    @assert w_ ≈ w
    @assert sum(w_) ≈ 2.0
 
    # Test case: [0, 1]
    x_, w_ = adjust_xw(x, w, 0.0, 1.0) 
    @assert x_ ≈ 0.5 .* x .+ 0.5
    @assert sum(w_) ≈ 1.0
 end

In [18]:
# 被積分関数の定義
f(x) = exp(x)

# 数値積分の計算 (.はbroadcast, dotはベクトル同士の内積)
x_adjusted, w_adjusted = adjust_xw(x, w, a, b)
integral = dot(f.(x_adjusted), w_adjusted)      #'f.': fについて全成分を書き出す？

println("Approximated integral: ", integral)

Approximated integral: 1.718281828459045


N = 5       1.7182818284583914  
N = 10      1.7182818284590455  
N = 20      1.7182818284590444  
N = 100     1.718281828459045  

In [ ]:

err = []
N = []
exactsl = 2.0 

# 被積分関数の定義
f(x) = exp(x)
for i in Nmax
    # 数値積分の計算 (.はbroadcast, dotはベクトル同士の内積)
    x_adjusted, w_adjusted = adjust_xw(x, w, a, b)
    integral = dot(f.(x_adjusted), w_adjusted)      #'f.': fについて全成分を書き出す？

    N += [i]
    err += [abs(integral - exactsl)]
end

plot(N, err)

In [21]:
for i=1,10
    println(i)
end

LoadError: syntax: invalid iteration specification

In [ ]:
mutable struct WrapFunc
    f
    sampling_points::Vector{Tuple{Float64,Float64}}
end
function (w::WrapFunc)(kx, ky)
    push!(w.sampling_points, (kx, ky))
    return w.f(kx, ky)
end
gk_ = WrapFunc(gk, [])
@show length(gk_.sampling_points)
gk_(0.0, 0.0)
@show length(gk_.sampling_points)
gk_(0.0, 0.0)
@show length(gk_.sampling_points)